In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/matches-brazilian-football-from-2003-to-2019/ranking-2003-2021.txt
/kaggle/input/matches-brazilian-football-from-2003-to-2019/average-attendance-2003-2019.txt
/kaggle/input/matches-brazilian-football-from-2003-to-2019/matches-2003-2021.txt


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df_matches = pd.read_csv('../input/matches-brazilian-football-from-2003-to-2019/matches-2003-2021.txt', delimiter=';', encoding='latin-1')

In [4]:
df_matches

,#ROUND,DATE,HOUR,HOME-TEAM,SCORE,AWAY-TEAM
0,1,29/03/2003,18:00,Guarani/SP,4x2,Vasco/RJ
1,1,29/03/2003,18:00,Atlético/PR,2x0,Grêmio/RS
2,1,30/03/2003,18:00,Goiás/GO,2x2,Paysandu/PA
3,1,30/03/2003,18:00,Corinthians/SP,0x3,Atlético/MG
4,1,30/03/2003,16:00,Flamengo/RJ,1x1,Coritiba/PR
...,...,...,...,...,...,...
7641,2,09/12/2021,21:30,Grêmio/RS,4x3,Atlético/MG
7642,2,09/12/2021,21:30,Fortaleza/CE,2x1,Bahia/BA
7643,2,09/12/2021,21:30,Sport/PE,1x1,Atlético/PR
7644,2,09/12/2021,21:30,Bragantino/SP,1x0,Internacional/RS


In [5]:
df_matches.drop(columns=['#ROUND', 'HOUR'], inplace=True)

In [6]:
currentYear = datetime.now().year

In [7]:
for index, row in enumerate(df_matches['DATE']):
    if int(row[6:10]) < currentYear:
        df_matches.drop(index, axis=0, inplace=True)

In [8]:
df_matches.head()

,DATE,HOME-TEAM,SCORE,AWAY-TEAM
7266,30/05/2021,Internacional/RS,2x2,Sport/PE
7267,30/05/2021,Atlético/PR,1x0,América/MG
7268,30/05/2021,Chapecoense/SC,0x3,Bragantino/SP
7269,30/05/2021,Corinthians/SP,0x1,Atlético/GO
7270,30/05/2021,Ceará/CE,3x2,Grêmio/RS


In [9]:
df_matches.drop(columns=['DATE'], inplace=True)

In [10]:
df_matches.tail()

,HOME-TEAM,SCORE,AWAY-TEAM
7641,Grêmio/RS,4x3,Atlético/MG
7642,Fortaleza/CE,2x1,Bahia/BA
7643,Sport/PE,1x1,Atlético/PR
7644,Bragantino/SP,1x0,Internacional/RS
7645,Juventude/RS,1x0,Corinthians/SP


In [11]:
df_matches[['SCORE-HOME-TEAM', 'SCORE-AWAY-TEAM']] = df_matches['SCORE'].str.split('x', expand=True)

In [12]:
df_matches.head()

,HOME-TEAM,SCORE,AWAY-TEAM,SCORE-HOME-TEAM,SCORE-AWAY-TEAM
7266,Internacional/RS,2x2,Sport/PE,2,2
7267,Atlético/PR,1x0,América/MG,1,0
7268,Chapecoense/SC,0x3,Bragantino/SP,0,3
7269,Corinthians/SP,0x1,Atlético/GO,0,1
7270,Ceará/CE,3x2,Grêmio/RS,3,2


In [13]:
win_home_team = np.where(df_matches['SCORE-HOME-TEAM'] > df_matches['SCORE-AWAY-TEAM'], 1, 0)
df_matches['WIN-HOME-TEAM'] = win_home_team

In [14]:
df_matches.head(10)

,HOME-TEAM,SCORE,AWAY-TEAM,SCORE-HOME-TEAM,SCORE-AWAY-TEAM,WIN-HOME-TEAM
7266,Internacional/RS,2x2,Sport/PE,2,2,0
7267,Atlético/PR,1x0,América/MG,1,0,1
7268,Chapecoense/SC,0x3,Bragantino/SP,0,3,0
7269,Corinthians/SP,0x1,Atlético/GO,0,1,0
7270,Ceará/CE,3x2,Grêmio/RS,3,2,1
7271,Flamengo/RJ,1x0,Palmeiras/SP,1,0,1
7272,Atlético/MG,1x2,Fortaleza/CE,1,2,0
7273,São Paulo/SP,0x0,Fluminense/RJ,0,0,0
7274,Bahia/BA,3x0,Santos/SP,3,0,1
7275,Cuiabá/MT,2x2,Juventude/RS,2,2,0


In [15]:
df_matches.drop(columns=['SCORE', 'SCORE-HOME-TEAM', 'SCORE-AWAY-TEAM'], inplace=True)

In [16]:
df_matches

,HOME-TEAM,AWAY-TEAM,WIN-HOME-TEAM
7266,Internacional/RS,Sport/PE,0
7267,Atlético/PR,América/MG,1
7268,Chapecoense/SC,Bragantino/SP,0
7269,Corinthians/SP,Atlético/GO,0
7270,Ceará/CE,Grêmio/RS,1
...,...,...,...
7641,Grêmio/RS,Atlético/MG,1
7642,Fortaleza/CE,Bahia/BA,1
7643,Sport/PE,Atlético/PR,0
7644,Bragantino/SP,Internacional/RS,1


In [17]:
df_matches.reset_index(drop=True, inplace=True)

In [18]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
col_1 = df_matches['HOME-TEAM']
set_col_1 = list(set(col_1))
le.fit(col_1)
dict_home_teams = dict(zip(set_col_1, le.transform(set_col_1)))

In [19]:
df_home_teams_le = pd.DataFrame(dict_home_teams.items(), columns=['TEAM', 'LABEL-ENCODING'])

In [20]:
df_home_teams_le

,TEAM,LABEL-ENCODING
0,Internacional/RS,14
1,Atlético/PR,3
2,Ceará/CE,6
3,Palmeiras/SP,16
4,Sport/PE,18
5,Fortaleza/CE,12
6,Cuiabá/MT,9
7,Juventude/RS,15
8,Fluminense/RJ,11
9,Corinthians/SP,8


In [21]:
le = preprocessing.LabelEncoder()
col_2 = df_matches['AWAY-TEAM']
set_col_2 = list(set(col_2))
le.fit(col_2)
dict_away_teams = dict(zip(set_col_2, le.transform(set_col_2)))

In [22]:
df_away_teams_le = pd.DataFrame(dict_away_teams.items(), columns=['TEAM', 'LABEL-ENCODING'])

In [23]:
df_away_teams_le

,TEAM,LABEL-ENCODING
0,Internacional/RS,14
1,Atlético/PR,3
2,Ceará/CE,6
3,Palmeiras/SP,16
4,Sport/PE,18
5,Fortaleza/CE,12
6,Cuiabá/MT,9
7,Fluminense/RJ,11
8,Juventude/RS,15
9,Corinthians/SP,8


In [24]:
df_teams_le = df_home_teams_le.merge(df_away_teams_le, how='inner', on='TEAM')

In [25]:
df_teams_le

,TEAM,LABEL-ENCODING_x,LABEL-ENCODING_y
0,Internacional/RS,14,14
1,Atlético/PR,3,3
2,Ceará/CE,6,6
3,Palmeiras/SP,16,16
4,Sport/PE,18,18
5,Fortaleza/CE,12,12
6,Cuiabá/MT,9,9
7,Juventude/RS,15,15
8,Fluminense/RJ,11,11
9,Corinthians/SP,8,8


In [26]:
df_teams_le.drop(columns=['LABEL-ENCODING_y'], inplace=True)

In [27]:
df_teams_le.rename(columns={'LABEL-ENCODING_x': 'LABEL-ENCODING'}, inplace=True)

In [28]:
df_teams_le

,TEAM,LABEL-ENCODING
0,Internacional/RS,14
1,Atlético/PR,3
2,Ceará/CE,6
3,Palmeiras/SP,16
4,Sport/PE,18
5,Fortaleza/CE,12
6,Cuiabá/MT,9
7,Juventude/RS,15
8,Fluminense/RJ,11
9,Corinthians/SP,8


In [29]:
for k, team in enumerate(df_teams_le['TEAM']):
    df_matches.loc[df_matches['HOME-TEAM'] == team, 'HOME-TEAM'] =  df_teams_le['LABEL-ENCODING'].iloc[k]

In [30]:
for k, team in enumerate(df_teams_le['TEAM']):
    df_matches.loc[df_matches['AWAY-TEAM'] == team, 'AWAY-TEAM'] =  df_teams_le['LABEL-ENCODING'].iloc[k]

In [31]:
display(df_matches)

,HOME-TEAM,AWAY-TEAM,WIN-HOME-TEAM
0,14,18,0
1,3,0,1
2,7,5,0
3,8,1,0
4,6,13,1
...,...,...,...
375,13,2,1
376,12,4,1
377,18,3,0
378,5,14,1


In [32]:
# Predictors and target
y = df_matches['WIN-HOME-TEAM']
x = df_matches.drop('WIN-HOME-TEAM', axis = 1)

In [33]:
x.shape

(380, 2)

In [34]:
y.shape

(380,)

In [35]:
from sklearn.model_selection import train_test_split

# Training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

In [36]:
from sklearn.ensemble import ExtraTreesClassifier

# Model
model = ExtraTreesClassifier()
model.fit(x_train, y_train)

# Printing result
result = model.score(x_test, y_test)
print('Precision:', result)

Precision: 0.5175438596491229


In [37]:
y_test[0:5]

172    0
145    1
103    0
94     1
274    0
Name: WIN-HOME-TEAM, dtype: int64

In [38]:
x_test[0:5]

,HOME-TEAM,AWAY-TEAM
172,5,2
145,0,11
103,15,1
94,2,10
274,14,8


In [39]:
prediction = model.predict(x_test[0:5])

In [40]:
prediction

array([0, 1, 0, 1, 0])